In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [26]:
import ee
import pandas as pd
import numpy as np
import datetime
import os
from dateutil.relativedelta import relativedelta
from google.colab import files
import matplotlib.pyplot as plt
import folium

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [27]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gen-lang-client-0253961861')

In [ ]:
def parse_date(date_str):
    """Convierte una cadena de fecha en formato YYYY-MM-DD a objeto datetime."""
    try:
        return datetime.datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        raise ValueError("El formato de fecha debe ser YYYY-MM-DD")

def get_date_ranges(start_date, end_date, frequency):
    """
    Genera rangos de fechas según la frecuencia especificada.

    Args:
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin
        frequency (str): 'daily', 'monthly', o 'annual'

    Returns:
        list: Lista de tuplas (fecha_inicio, fecha_fin) para cada período
    """
    date_ranges = []

    if frequency == 'daily':
        current = start_date
        while current <= end_date:
            date_str = current.strftime('%Y-%m-%d')
            date_ranges.append((date_str, date_str))
            current += datetime.timedelta(days=1)

    elif frequency == 'monthly':
        current = datetime.datetime(start_date.year, start_date.month, 1)
        while current <= end_date:
            next_month = current + relativedelta(months=1)
            end_of_month = (next_month - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
            date_ranges.append((current.strftime('%Y-%m-%d'), end_of_month))
            current = next_month

    elif frequency == 'annual':
        current_year = start_date.year
        while current_year <= end_date.year:
            year_start = datetime.datetime(current_year, 1, 1).strftime('%Y-%m-%d')
            year_end = datetime.datetime(current_year, 12, 31).strftime('%Y-%m-%d')
            date_ranges.append((year_start, year_end))
            current_year += 1

    return date_ranges

def get_albedo_data(point, start_date, end_date):
    """
    Extrae datos de albedo (black-sky y white-sky) para un punto y período específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        ee.ImageCollection: Colección de imágenes con datos de albedo
    """
    albedo_collection = ee.ImageCollection('MODIS/061/MCD43A3') \
        .filterDate(start_date, end_date) \
        .select(['Albedo_BSA_vis', 'Albedo_WSA_vis', 'Albedo_BSA_nir', 'Albedo_WSA_nir', 'Albedo_BSA_shortwave', 'Albedo_WSA_shortwave'])

    return albedo_collection

def get_solar_radiation_data(point, start_date, end_date):
    """
    Extrae datos de radiación solar para un punto y período específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        ee.ImageCollection: Colección de imágenes con datos de radiación solar
    """
    radiation_collection = ee.ImageCollection('MODIS/061/MCD18A1') \
        .filterDate(start_date, end_date) \
        .select(['Incident_Shortwave_Radiation_Daily_Mean', 'Incident_Shortwave_Radiation_Daily_Total'])

    return radiation_collection

def get_temperature_data(point, start_date, end_date):
    """
    Extrae datos de temperatura superficial para un punto y período específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        ee.ImageCollection: Colección de imágenes con datos de temperatura
    """
    temperature_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
        .filterDate(start_date, end_date) \
        .select(['LST_Day_1km', 'LST_Night_1km', 'QC_Day', 'QC_Night'])

    return temperature_collection

def get_wind_data(point, start_date, end_date):
    """
    Extrae datos de viento para un punto y período específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD

    Returns:
        ee.ImageCollection: Colección de imágenes con datos de viento
    """
    # Obtener datos de viento horarios
    wind_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
        .filterDate(start_date, end_date) \
        .select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])

    # Método alternativo para agrupar por día sin usar startOfDay()
    # Usamos format() para truncar la hora y obtener solo la fecha
    def add_date_band(image):
        # Obtener la fecha como string en formato YYYY-MM-DD
        date_string = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        # Añadir como propiedad
        return image.set('date_string', date_string)

    # Añadir la propiedad de fecha a cada imagen
    wind_collection_with_date = wind_collection.map(add_date_band)

    # Obtener lista de fechas únicas como strings
    unique_dates = wind_collection_with_date.aggregate_array('date_string').distinct()

    # Función para calcular la media diaria
    def calculate_daily_mean(date_string):
        # Filtrar imágenes para esta fecha
        daily_images = wind_collection_with_date.filter(ee.Filter.eq('date_string', date_string))
        # Calcular la media
        mean_image = daily_images.mean()
        # Convertir la fecha string a timestamp
        timestamp = ee.Date(date_string).millis()
        # Añadir timestamp y fecha string como propiedades
        return mean_image.set('system:time_start', timestamp).set('date_string', date_string)

    # Calcular medias diarias
    daily_wind = ee.ImageCollection.fromImages(unique_dates.map(calculate_daily_mean))

    return daily_wind

def get_elevation_data(point):
    """
    Extrae datos de elevación y variables topográficas derivadas para un punto específico.

    Args:
        point (ee.Geometry.Point): Punto de interés

    Returns:
        ee.Image: Imagen con datos de elevación y variables derivadas
    """
    # Obtener datos de elevación SRTM
    elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')

    # Calcular pendiente y aspecto (orientación)
    slope = ee.Terrain.slope(elevation)
    aspect = ee.Terrain.aspect(elevation)

    # Combinar en una sola imagen
    topo = elevation.addBands(slope).addBands(aspect).rename(['elevation', 'slope', 'aspect'])

    return topo

def get_landcover_data(point, year):
    """
    Extrae datos de cobertura terrestre para un punto y año específicos.

    Args:
        point (ee.Geometry.Point): Punto de interés
        year (int): Año para el cual obtener la cobertura terrestre

    Returns:
        ee.Image: Imagen con datos de cobertura terrestre
    """
    # Usar el producto de cobertura terrestre MODIS
    landcover_collection = ee.ImageCollection('MODIS/006/MCD12Q1')

    # Filtrar por el año especificado
    year_start = f"{year}-01-01"
    year_end = f"{year}-12-31"
    landcover = landcover_collection.filterDate(year_start, year_end).first().select('LC_Type1')

    return landcover

def extract_point_values(image_collection, point, scale=500, band_name=None):
    """
    Extrae valores para un punto específico de una colección de imágenes.

    Args:
        image_collection (ee.ImageCollection): Colección de imágenes
        point (ee.Geometry.Point): Punto de interés
        scale (int): Escala en metros
        band_name (str, optional): Nombre de la banda específica a extraer

    Returns:
        list: Lista de diccionarios con fecha y valores
    """
    # Verificar si la colección está vacía
    count = image_collection.size().getInfo()
    if count == 0:
        print("⚠️ Advertencia: Colección de imágenes vacía")
        return []

    def extract_from_image(image):
        # Si se especifica una banda, seleccionarla
        if band_name:
            image = image.select(band_name)

        # Extraer valor en el punto
        value = image.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=scale
        )

        # Crear feature con fecha y valor
        return ee.Feature(None, {
            'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
            'values': value
        })

    # Aplicar la función a cada imagen en la colección
    features = image_collection.map(extract_from_image)

    # Convertir a lista para descargar
    return features.aggregate_array('properties').getInfo()

def extract_static_values(image, point, scale=500):
    """
    Extrae valores para un punto específico de una imagen estática.

    Args:
        image (ee.Image): Imagen
        point (ee.Geometry.Point): Punto de interés
        scale (int): Escala en metros

    Returns:
        dict: Diccionario con valores
    """
    values = image.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=scale
    ).getInfo()

    return values

def aggregate_data(data_list, frequency, start_date, end_date):
    """
    Agrega datos según la frecuencia especificada.

    Args:
        data_list (list): Lista de diccionarios con datos
        frequency (str): 'daily', 'monthly', o 'annual'
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin

    Returns:
        list: Lista de diccionarios con datos agregados
    """
    if not data_list:
        return []

    # Convertir a DataFrame
    df = pd.DataFrame(data_list)

    # Asegurarse de que la columna de fecha es datetime
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    # Agregar según frecuencia
    if frequency == 'daily':
        return df.reset_index().to_dict('records')
    elif frequency == 'monthly':
        monthly_df = df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        annual_df = df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def process_albedo_data(albedo_collection, point, frequency, start_date, end_date):
    """
    Procesa datos de albedo y los agrega según la frecuencia especificada.

    Args:
        albedo_collection (ee.ImageCollection): Colección de imágenes de albedo
        point (ee.Geometry.Point): Punto de interés
        frequency (str): 'daily', 'monthly', o 'annual'
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin

    Returns:
        list: Lista de diccionarios con datos de albedo agregados
    """
    # Extraer valores para cada banda
    bsa_vis_data = extract_point_values(albedo_collection.select('Albedo_BSA_vis'), point)
    wsa_vis_data = extract_point_values(albedo_collection.select('Albedo_WSA_vis'), point)
    bsa_nir_data = extract_point_values(albedo_collection.select('Albedo_BSA_nir'), point)
    wsa_nir_data = extract_point_values(albedo_collection.select('Albedo_WSA_nir'), point)
    bsa_sw_data = extract_point_values(albedo_collection.select('Albedo_BSA_shortwave'), point)
    wsa_sw_data = extract_point_values(albedo_collection.select('Albedo_WSA_shortwave'), point)

    # Verificar si hay datos disponibles
    if not bsa_vis_data and not wsa_vis_data and not bsa_nir_data and not wsa_nir_data and not bsa_sw_data and not wsa_sw_data:
        print("⚠️ Advertencia: No hay datos de albedo disponibles para el período seleccionado")
        return []

    # Convertir a DataFrames
    dfs = []

    if bsa_vis_data:
        bsa_vis_df = pd.DataFrame([{'date': item['date'], 'bsa_vis': item['values'].get('Albedo_BSA_vis', None)} for item in bsa_vis_data])
        # Convertir valores a escala correcta (dividir por 1000)
        bsa_vis_df['bsa_vis'] = bsa_vis_df['bsa_vis'].apply(lambda x: x / 1000.0 if x is not None else None)
        dfs.append(bsa_vis_df)

    if wsa_vis_data:
        wsa_vis_df = pd.DataFrame([{'date': item['date'], 'wsa_vis': item['values'].get('Albedo_WSA_vis', None)} for item in wsa_vis_data])
        wsa_vis_df['wsa_vis'] = wsa_vis_df['wsa_vis'].apply(lambda x: x / 1000.0 if x is not None else None)
        dfs.append(wsa_vis_df)

    if bsa_nir_data:
        bsa_nir_df = pd.DataFrame([{'date': item['date'], 'bsa_nir': item['values'].get('Albedo_BSA_nir', None)} for item in bsa_nir_data])
        bsa_nir_df['bsa_nir'] = bsa_nir_df['bsa_nir'].apply(lambda x: x / 1000.0 if x is not None else None)
        dfs.append(bsa_nir_df)

    if wsa_nir_data:
        wsa_nir_df = pd.DataFrame([{'date': item['date'], 'wsa_nir': item['values'].get('Albedo_WSA_nir', None)} for item in wsa_nir_data])
        wsa_nir_df['wsa_nir'] = wsa_nir_df['wsa_nir'].apply(lambda x: x / 1000.0 if x is not None else None)
        dfs.append(wsa_nir_df)

    if bsa_sw_data:
        bsa_sw_df = pd.DataFrame([{'date': item['date'], 'bsa_sw': item['values'].get('Albedo_BSA_shortwave', None)} for item in bsa_sw_data])
        bsa_sw_df['bsa_sw'] = bsa_sw_df['bsa_sw'].apply(lambda x: x / 1000.0 if x is not None else None)
        dfs.append(bsa_sw_df)

    if wsa_sw_data:
        wsa_sw_df = pd.DataFrame([{'date': item['date'], 'wsa_sw': item['values'].get('Albedo_WSA_shortwave', None)} for item in wsa_sw_data])
        wsa_sw_df['wsa_sw'] = wsa_sw_df['wsa_sw'].apply(lambda x: x / 1000.0 if x is not None else None)
        dfs.append(wsa_sw_df)

    # Si no hay DataFrames, retornar lista vacía
    if not dfs:
        return []

    # Combinar DataFrames
    albedo_df = dfs[0]
    for df in dfs[1:]:
        albedo_df = pd.merge(albedo_df, df, on='date', how='outer')

    # Convertir fechas a datetime
    albedo_df['date'] = pd.to_datetime(albedo_df['date'])

    # Agregar según frecuencia
    if frequency == 'daily':
        return albedo_df.to_dict('records')
    elif frequency == 'monthly':
        albedo_df.set_index('date', inplace=True)
        monthly_df = albedo_df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        albedo_df.set_index('date', inplace=True)
        annual_df = albedo_df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def process_radiation_data(radiation_collection, point, frequency, start_date, end_date):
    """
    Procesa datos de radiación solar y los agrega según la frecuencia especificada.

    Args:
        radiation_collection (ee.ImageCollection): Colección de imágenes de radiación
        point (ee.Geometry.Point): Punto de interés
        frequency (str): 'daily', 'monthly', o 'annual'
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin

    Returns:
        list: Lista de diccionarios con datos de radiación agregados
    """
    # Extraer valores para cada banda
    mean_data = extract_point_values(radiation_collection.select('Incident_Shortwave_Radiation_Daily_Mean'), point)
    total_data = extract_point_values(radiation_collection.select('Incident_Shortwave_Radiation_Daily_Total'), point)

    # Verificar si hay datos disponibles
    if not mean_data and not total_data:
        print("⚠️ Advertencia: No hay datos de radiación solar disponibles para el período seleccionado")
        return []

    # Convertir a DataFrames
    dfs = []

    if mean_data:
        mean_df = pd.DataFrame([{'date': item['date'], 'radiation_mean': item['values'].get('Incident_Shortwave_Radiation_Daily_Mean', None)} for item in mean_data])
        dfs.append(mean_df)

    if total_data:
        total_df = pd.DataFrame([{'date': item['date'], 'radiation_total': item['values'].get('Incident_Shortwave_Radiation_Daily_Total', None)} for item in total_data])
        dfs.append(total_df)

    # Si no hay DataFrames, retornar lista vacía
    if not dfs:
        return []

    # Combinar DataFrames
    radiation_df = dfs[0]
    for df in dfs[1:]:
        radiation_df = pd.merge(radiation_df, df, on='date', how='outer')

    # Convertir fechas a datetime
    radiation_df['date'] = pd.to_datetime(radiation_df['date'])

    # Agregar según frecuencia
    if frequency == 'daily':
        return radiation_df.to_dict('records')
    elif frequency == 'monthly':
        radiation_df.set_index('date', inplace=True)
        monthly_df = radiation_df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        radiation_df.set_index('date', inplace=True)
        annual_df = radiation_df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def process_temperature_data(temperature_collection, point, frequency, start_date, end_date):
    """
    Procesa datos de temperatura y los agrega según la frecuencia especificada.

    Args:
        temperature_collection (ee.ImageCollection): Colección de imágenes de temperatura
        point (ee.Geometry.Point): Punto de interés
        frequency (str): 'daily', 'monthly', o 'annual'
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin

    Returns:
        list: Lista de diccionarios con datos de temperatura agregados
    """
    # Extraer valores para cada banda
    day_data = extract_point_values(temperature_collection.select('LST_Day_1km'), point)
    night_data = extract_point_values(temperature_collection.select('LST_Night_1km'), point)

    # Verificar si hay datos disponibles
    if not day_data and not night_data:
        print("⚠️ Advertencia: No hay datos de temperatura disponibles para el período seleccionado")
        return []

    # Convertir a DataFrames
    dfs = []

    if day_data:
        day_df = pd.DataFrame([{'date': item['date'], 'temp_day': item['values'].get('LST_Day_1km', None)} for item in day_data])
        day_df['temp_day'] = day_df['temp_day'].apply(lambda x: (x * 0.02) - 273.15 if x is not None else None)
        dfs.append(day_df)

    if night_data:
        night_df = pd.DataFrame([{'date': item['date'], 'temp_night': item['values'].get('LST_Night_1km', None)} for item in night_data])
        night_df['temp_night'] = night_df['temp_night'].apply(lambda x: (x * 0.02) - 273.15 if x is not None else None)
        dfs.append(night_df)

    # Si no hay DataFrames, retornar lista vacía
    if not dfs:
        return []

    # Combinar DataFrames
    temp_df = dfs[0]
    for df in dfs[1:]:
        temp_df = pd.merge(temp_df, df, on='date', how='outer')

    # Añadir temperatura media si hay datos de día y noche
    if 'temp_day' in temp_df.columns and 'temp_night' in temp_df.columns:
        temp_df['temp_mean'] = temp_df[['temp_day', 'temp_night']].mean(axis=1)
    elif 'temp_day' in temp_df.columns:
        temp_df['temp_mean'] = temp_df['temp_day']
    elif 'temp_night' in temp_df.columns:
        temp_df['temp_mean'] = temp_df['temp_night']

    # Convertir fechas a datetime
    temp_df['date'] = pd.to_datetime(temp_df['date'])

    # Agregar según frecuencia
    if frequency == 'daily':
        return temp_df.to_dict('records')
    elif frequency == 'monthly':
        temp_df.set_index('date', inplace=True)
        monthly_df = temp_df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        temp_df.set_index('date', inplace=True)
        annual_df = temp_df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def process_wind_data(wind_collection, point, frequency, start_date, end_date):
    """
    Procesa datos de viento y los agrega según la frecuencia especificada.

    Args:
        wind_collection (ee.ImageCollection): Colección de imágenes de viento
        point (ee.Geometry.Point): Punto de interés
        frequency (str): 'daily', 'monthly', o 'annual'
        start_date (datetime): Fecha de inicio
        end_date (datetime): Fecha de fin

    Returns:
        list: Lista de diccionarios con datos de viento agregados
    """
    # Extraer valores para cada componente
    u_data = extract_point_values(wind_collection.select('u_component_of_wind_10m'), point)
    v_data = extract_point_values(wind_collection.select('v_component_of_wind_10m'), point)

    # Verificar si hay datos disponibles
    if not u_data and not v_data:
        print("⚠️ Advertencia: No hay datos de viento disponibles para el período seleccionado")
        return []

    # Convertir a DataFrames
    dfs = []

    if u_data:
        u_df = pd.DataFrame([{'date': item['date'], 'wind_u': item['values'].get('u_component_of_wind_10m', None)} for item in u_data])
        dfs.append(u_df)

    if v_data:
        v_df = pd.DataFrame([{'date': item['date'], 'wind_v': item['values'].get('v_component_of_wind_10m', None)} for item in v_data])
        dfs.append(v_df)

    # Si no hay DataFrames, retornar lista vacía
    if not dfs:
        return []

    # Combinar DataFrames
    wind_df = dfs[0]
    for df in dfs[1:]:
        wind_df = pd.merge(wind_df, df, on='date', how='outer')

    # Calcular velocidad y dirección del viento si hay ambos componentes
    if 'wind_u' in wind_df.columns and 'wind_v' in wind_df.columns:
        wind_df['wind_speed'] = wind_df.apply(lambda row: np.sqrt(row['wind_u']**2 + row['wind_v']**2) if pd.notnull(row['wind_u']) and pd.notnull(row['wind_v']) else None, axis=1)
        wind_df['wind_direction'] = wind_df.apply(lambda row: (270 - np.degrees(np.arctan2(row['wind_v'], row['wind_u']))) % 360 if pd.notnull(row['wind_u']) and pd.notnull(row['wind_v']) else None, axis=1)

    # Convertir fechas a datetime
    wind_df['date'] = pd.to_datetime(wind_df['date'])

    # Agregar según frecuencia
    if frequency == 'daily':
        return wind_df.to_dict('records')
    elif frequency == 'monthly':
        wind_df.set_index('date', inplace=True)
        monthly_df = wind_df.resample('M').mean()
        monthly_df.index = monthly_df.index.strftime('%Y-%m-%d')
        return monthly_df.reset_index().to_dict('records')
    elif frequency == 'annual':
        wind_df.set_index('date', inplace=True)
        annual_df = wind_df.resample('Y').mean()
        annual_df.index = annual_df.index.strftime('%Y-%m-%d')
        return annual_df.reset_index().to_dict('records')

def merge_all_data(albedo_data, radiation_data, temperature_data, wind_data, topo_data, landcover_data):
    """
    Combina todos los datos en un solo DataFrame.

    Args:
        albedo_data (list): Lista de diccionarios con datos de albedo
        radiation_data (list): Lista de diccionarios con datos de radiación
        temperature_data (list): Lista de diccionarios con datos de temperatura
        wind_data (list): Lista de diccionarios con datos de viento
        topo_data (dict): Diccionario con datos topográficos
        landcover_data (dict): Diccionario con datos de cobertura terrestre

    Returns:
        pd.DataFrame: DataFrame con todos los datos combinados
    """
    # Convertir listas a DataFrames
    albedo_df = pd.DataFrame(albedo_data) if albedo_data else pd.DataFrame()
    radiation_df = pd.DataFrame(radiation_data) if radiation_data else pd.DataFrame()
    temperature_df = pd.DataFrame(temperature_data) if temperature_data else pd.DataFrame()
    wind_df = pd.DataFrame(wind_data) if wind_data else pd.DataFrame()

    # Combinar DataFrames
    dfs = [df for df in [albedo_df, radiation_df, temperature_df, wind_df] if not df.empty]

    if not dfs:
        print("⚠️ Advertencia: No hay datos disponibles para el período seleccionado")
        return pd.DataFrame({'date': []})

    merged_df = dfs[0]
    for df in dfs[1:]:
        if not df.empty and 'date' in df.columns and 'date' in merged_df.columns:
            merged_df = pd.merge(merged_df, df, on='date', how='outer')

    # Añadir datos topográficos (constantes para todas las fechas)
    if topo_data:
        for key, value in topo_data.items():
            merged_df[key] = value

    # Añadir datos de cobertura terrestre (constantes para todas las fechas)
    if landcover_data:
        for key, value in landcover_data.items():
            merged_df[key] = value

    return merged_df

def visualize_location(lat, lon, zoom=10):
    """
    Visualiza la ubicación en un mapa interactivo.

    Args:
        lat (float): Latitud
        lon (float): Longitud
        zoom (int): Nivel de zoom
    """
    # Crear mapa centrado en la ubicación
    m = folium.Map(location=[lat, lon], zoom_start=zoom)

    # Añadir marcador
    folium.Marker(
        location=[lat, lon],
        popup=f"Lat: {lat}, Lon: {lon}",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(m)

    # Mostrar mapa
    display(m)

def plot_data(df, variable, title=None, ylabel=None):
    """
    Genera un gráfico de línea para una variable específica.

    Args:
        df (pd.DataFrame): DataFrame con los datos
        variable (str): Nombre de la columna a graficar
        title (str, optional): Título del gráfico
        ylabel (str, optional): Etiqueta del eje Y
    """
    if df.empty or variable not in df.columns:
        print(f"⚠️ No hay datos disponibles para graficar {variable}")
        return

    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df[variable], marker='o', linestyle='-')
    plt.title(title or f"{variable} a lo largo del tiempo")
    plt.xlabel("Fecha")
    plt.ylabel(ylabel or variable)
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def extract_solar_data_colab(lat, lon, start_date, end_date, frequency='monthly', output_filename='solar_data.csv'):
    """
    Función principal para extraer datos solares en Google Colab.

    Args:
        lat (float): Latitud del punto de interés
        lon (float): Longitud del punto de interés
        start_date (str): Fecha de inicio en formato YYYY-MM-DD
        end_date (str): Fecha de fin en formato YYYY-MM-DD
        frequency (str): Frecuencia de los datos ('daily', 'monthly', o 'annual')
        output_filename (str): Nombre del archivo CSV de salida

    Returns:
        pd.DataFrame: DataFrame con todos los datos combinados
    """
    # Inicializar Earth Engine
    if not initialize_ee_colab():
        return None

    # Convertir fechas
    start_date_dt = parse_date(start_date)
    end_date_dt = parse_date(end_date)

    # Crear punto
    point = ee.Geometry.Point([lon, lat])

    # Visualizar ubicación
    print(f"📍 Ubicación seleccionada: Latitud {lat}, Longitud {lon}")
    visualize_location(lat, lon)

    print(f"⏳ Extrayendo datos desde {start_date} hasta {end_date} con frecuencia {frequency}...")

    try:
        # Obtener datos
        print("🔄 Obteniendo datos de albedo...")
        albedo_collection = get_albedo_data(point, start_date, end_date)

        print("🔄 Obteniendo datos de radiación solar...")
        radiation_collection = get_solar_radiation_data(point, start_date, end_date)

        print("🔄 Obteniendo datos de temperatura...")
        temperature_collection = get_temperature_data(point, start_date, end_date)

        print("🔄 Obteniendo datos de viento...")
        wind_collection = get_wind_data(point, start_date, end_date)

        print("🔄 Obteniendo datos topográficos...")
        topo_data = get_elevation_data(point)

        # Para cobertura terrestre, usar el año medio del rango
        mid_year = start_date_dt.year + (end_date_dt.year - start_date_dt.year) // 2
        print(f"🔄 Obteniendo datos de cobertura terrestre para el año {mid_year}...")
        landcover_data = get_landcover_data(point, mid_year)

        # Procesar datos
        print("🔄 Procesando datos de albedo...")
        albedo_data = process_albedo_data(albedo_collection, point, frequency, start_date_dt, end_date_dt)

        print("🔄 Procesando datos de radiación solar...")
        radiation_data = process_radiation_data(radiation_collection, point, frequency, start_date_dt, end_date_dt)

        print("🔄 Procesando datos de temperatura...")
        temperature_data = process_temperature_data(temperature_collection, point, frequency, start_date_dt, end_date_dt)

        print("🔄 Procesando datos de viento...")
        wind_data = process_wind_data(wind_collection, point, frequency, start_date_dt, end_date_dt)

        print("🔄 Extrayendo datos topográficos...")
        topo_values = extract_static_values(topo_data, point)

        print("🔄 Extrayendo datos de cobertura terrestre...")
        landcover_values = extract_static_values(landcover_data, point)

        # Combinar todos los datos
        print("🔄 Combinando todos los datos...")
        merged_df = merge_all_data(albedo_data, radiation_data, temperature_data, wind_data, topo_values, landcover_values)

        if merged_df.empty:
            print("❌ No se encontraron datos para el período y ubicación especificados.")
            return pd.DataFrame({'date': []})

        # Guardar a CSV
        print(f"💾 Guardando datos en {output_filename}...")
        merged_df.to_csv(output_filename, index=False)

        # Descargar archivo
        files.download(output_filename)

        print(f"✅ ¡Completado! Los datos se han guardado y descargado como {output_filename}")

        # Mostrar algunas visualizaciones
        print("\n📊 Visualizaciones de los datos:")

        if 'radiation_mean' in merged_df.columns:
            plot_data(merged_df, 'radiation_mean', 'Radiación Solar Media Diaria', 'W/m²')

        if 'temp_mean' in merged_df.columns:
            plot_data(merged_df, 'temp_mean', 'Temperatura Media', '°C')

        if 'bsa_vis' in merged_df.columns:
            plot_data(merged_df, 'bsa_vis', 'Albedo Black-Sky (Visible)', 'Albedo')

        if 'wind_speed' in merged_df.columns:
            plot_data(merged_df, 'wind_speed', 'Velocidad del Viento', 'm/s')

        # Mostrar resumen de los datos
        if not merged_df.empty:
            print("\n📋 Resumen de los datos extraídos:")
            print(merged_df.describe())

        return merged_df

    except Exception as e:
        print(f"❌ Error durante la extracción de datos: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame({'date': []})



In [28]:
# Ejemplo de uso en Colab (descomenta para usar)
extract_solar_data_colab(
     lat=40.416775,
     lon=-3.703790,
     start_date='2022-01-01',
     end_date='2022-12-31',
     frequency='monthly',
     output_filename='madrid_2022_monthly.csv'
 )


Earth Engine inicializado correctamente.
✅ Autenticación completada con éxito.
📍 Ubicación seleccionada: Latitud 40.416775, Longitud -3.70379


⏳ Extrayendo datos desde 2022-01-01 hasta 2022-12-31 con frecuencia monthly...
🔄 Obteniendo datos de albedo...
🔄 Obteniendo datos de radiación solar...
🔄 Obteniendo datos de temperatura...
🔄 Obteniendo datos de viento...
🔄 Obteniendo datos topográficos...
🔄 Obteniendo datos de cobertura terrestre para el año 2022...
🔄 Procesando datos de albedo...


KeyError: 'date'

## Test the API

Test the API by printing the elevation of Mount Everest.

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

## Map visualization

`ee.Image` objects can be displayed to notebook output cells. The following two
examples demonstrate displaying a static image and an interactive map.


### Static image

The `IPython.display` module contains the `Image` function, which can display
the results of a URL representing an image generated from a call to the Earth
Engine `getThumbUrl` function. The following cell will display a thumbnail
of the global elevation model.

In [ ]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

### Interactive map

The [geemap](https://github.com/gee-community/geemap)
library can be used to display `ee.Image` objects on an interactive
[ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) map.

The following cell provides an example of using the `geemap.Map` object to
display an elevation model.

In [ ]:
# Import the geemap library.
import geemap

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a map object.
m = geemap.Map(center=[20, 0], zoom=3)

# Add the elevation model to the map object.
m.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Display the map.
display(m)

## Chart visualization

Some Earth Engine functions produce tabular data that can be plotted by
data visualization packages such as `matplotlib`. The following example
demonstrates the display of tabular data from Earth Engine as a scatter
plot. See [Charting in Colaboratory](https://colab.sandbox.google.com/notebooks/charts.ipynb)
for more information.

In [ ]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat TOA image.
img = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_034033_20000913')

# Select Red and NIR bands and sample 500 points.
samp_fc = img.select(['B3','B4']).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()